一篇文章可以看作是一串单词序列或者一串字符序列.
文本常见预处理步骤:
1. 将文本作为字符串加载到内存里
1. 将字符串拆分为词元(单词和字符)
2. 建立一个词表,将拆分的词元映射到数字索引
3. 将文本转为数字索引序列,方便模型操作

In [1]:
import re
import collections
import sys,os
sys.path.append(os.path.abspath("../"))
import d2l


/Users/zane/miniforge3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 下载数据集

In [2]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL+'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')
def read_time_machine():
    """讲时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'),'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+',' ',line).strip().lower() for line in lines]

lines = read_time_machine()
print(f"总行数:{len(lines)}") 
print(lines[0])
print(lines[-1])

总行数:3221
the time machine by h g wells
of man


### 词元化 Tokenize
词元 Token是文本的基本单位,可以是一个word,也可以是单个元素

In [3]:
def tokenize(lines,token="word"):
    """将每行文本拆分成单词或者字符词元"""
    
    if token in ("word", "char"):
        if token == "word":
            return [line.split() for line in lines]
        elif token == "char":
            return [list(line) for line in lines]
    else:
        assert("Error: Unknown Token: " + token)
        
lines1 = ["I am fine","Who are u"]
tokens1 = tokenize(lines1)
tokens2 = tokenize(lines1,token="char")
len(tokens1),tokens1,len(tokens2),tokens2

(2,
 [['I', 'am', 'fine'], ['Who', 'are', 'u']],
 2,
 [['I', ' ', 'a', 'm', ' ', 'f', 'i', 'n', 'e'],
  ['W', 'h', 'o', ' ', 'a', 'r', 'e', ' ', 'u']])

### 词表 Vocabulary Voca
词元的类型是字符串或char,而模型需要输入的是int型,因此我们需要构建一个字典,将词元映射到0开始的数字索引中.
思路:
1. 现将训练集中所有的文档合并在一起,对他们的唯一词元进行统计,得到的统计结果称为**语料(Corpus)**.
2. 根据每个唯一词元出现的频率进行索引分配.
3. 删除出现次数低于一定阈值的词元,降低复杂度.
4. Corpus中不存在或者已经删除的任何词元都将映射到一个特定的未知词元'\<unk>',同时增加一个列表,用于保存那些被保留的词元,例如填充词元'\<pad>',序列开始词元和序列结束词元'\<bos>','\<eos>'

In [4]:
class Vocab:
    """词表"""
    def __init__(self,tokens=None,min_freq=1,reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        
        # 按照出现的频率进行排序
        counter = self._count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key = lambda x:x[1],reverse= True)
        # print(self._token_freqs)
        

        self.idx_to_token = ['<unk>'] + reserved_tokens # 未知的索引为0
        self.token_to_idx ={
            token:idx 
            for idx,token in enumerate(self.idx_to_token)
        }

        for token,freq in self._token_freqs:
            # 开始向tokens中添加items
            # 过滤
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) -1
        
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self,tokens):
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_idx.get(tokens,self.unk)
        return [self.__getitem__[token] for token in tokens]
    
    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
        
    @property
    def unk(self):
        return 0
    
    @property
    def token_freqs(self):
        return self._token_freqs


    
        
    @staticmethod
    def _count_corpus(tokens):
        """统计词元频率

        Args:
            tokens (List): tokens是一维或者二维的list

        Returns:
            Counter: collections.Counter类
        """
        if len(tokens) ==0 or isinstance(tokens[0],list):
            tokens = [token for line in tokens for token in line ]
        return collections.Counter(tokens)



In [5]:
vocab = Vocab(tokenize(["asd abc","accbc ddddd"],token="char"))

### 模块整合

In [6]:
def load_corpus_time_machine(max_tokens = -1):
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines,"char")
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus,vocab

corpus,vocab = load_corpus_time_machine()
len(corpus),len(vocab),vocab.idx_to_token

(170580,
 28,
 ['<unk>',
  ' ',
  'e',
  't',
  'a',
  'i',
  'n',
  'o',
  's',
  'h',
  'r',
  'd',
  'l',
  'm',
  'u',
  'c',
  'f',
  'w',
  'g',
  'y',
  'p',
  'b',
  'v',
  'k',
  'x',
  'z',
  'j',
  'q'])